In [1]:
import pandas as pd
import numpy as np
import seaborn as sns  
import matplotlib.pyplot as plt
from datetime import date
from dateutil.relativedelta import relativedelta
%matplotlib inline 

# IMPORTANT:
# Redownload the files from slack and Outlook 

##### DATA (the anonymised dataframe) 

In [2]:
#import main dataframe
data = pd.read_csv('locations.csv', low_memory=False)

In [3]:
#rename City column and drop empty columns
data.rename( columns={'Unnamed: 5':'City'}, inplace=True )
data = data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1)

In [4]:
#drop VisitorID and ContentInfo
data = data.drop(columns=['VisitorID', 'ContentInfo'])

In [5]:
#just getting rid of the space after the name
data = data.replace('STH Ayshire St Leonards ', 'STH Ayshire St Leonards')
data.head()

,Time,ExternalID,Role,Building,City,LinkTitle,LinkType
0,29/10/2018 10:55,5325.0,User,Broom Ground,Stirling,Sam Smith,Internet
1,29/10/2018 10:50,5325.0,User,Broom Ground,Stirling,Elton John,Internet
2,29/10/2018 10:49,5325.0,User,Broom Ground,Stirling,My Music,Category
3,29/10/2018 10:49,5325.0,User,Broom Ground,Stirling,Entertainment,Category
4,29/10/2018 10:48,NaN,User,Belses Gardens - Care Home,Glasgow,BBC Formula 1,Internet


In [6]:
#drop nan values in Building
data = data.dropna(subset=['Building'])
data.head()

,Time,ExternalID,Role,Building,City,LinkTitle,LinkType
0,29/10/2018 10:55,5325.0,User,Broom Ground,Stirling,Sam Smith,Internet
1,29/10/2018 10:50,5325.0,User,Broom Ground,Stirling,Elton John,Internet
2,29/10/2018 10:49,5325.0,User,Broom Ground,Stirling,My Music,Category
3,29/10/2018 10:49,5325.0,User,Broom Ground,Stirling,Entertainment,Category
4,29/10/2018 10:48,NaN,User,Belses Gardens - Care Home,Glasgow,BBC Formula 1,Internet


In [7]:
#change time to datetime
data['Time'] = pd.to_datetime(data['Time'], format = "%d/%m/%Y %H:%M", utc=False)
data.head()

,Time,ExternalID,Role,Building,City,LinkTitle,LinkType
0,2018-10-29 10:55:00,5325.0,User,Broom Ground,Stirling,Sam Smith,Internet
1,2018-10-29 10:50:00,5325.0,User,Broom Ground,Stirling,Elton John,Internet
2,2018-10-29 10:49:00,5325.0,User,Broom Ground,Stirling,My Music,Category
3,2018-10-29 10:49:00,5325.0,User,Broom Ground,Stirling,Entertainment,Category
4,2018-10-29 10:48:00,NaN,User,Belses Gardens - Care Home,Glasgow,BBC Formula 1,Internet


In [8]:
#change ExternalIDs to integers
data.ExternalID = data.ExternalID.fillna(0).astype(int)

In [9]:
#drop rows that are identical
data = data.drop_duplicates(keep='first')

In [10]:
#new dataframe with no null ExternalID vaues
data = data[data.ExternalID != 0]

##### USERS (the user dataframe) 

In [11]:
#import users dataframe
users = pd.read_csv('PrimaryConditions age sex by customer reference.csv', low_memory=False)

In [12]:
#drop empty column and CleverCogsUserId
users = users.drop(users.columns[users.columns.str.contains('unnamed',case = False)],axis = 1)
users = users.drop(columns=['CleverCogsUserId'])

In [13]:
#delete empty row
users = users.dropna(axis=0, how='all', inplace=False)

In [14]:
#change ExternalID into integers
users['ExternalID'] = users['ExternalID'].apply(np.int64)
users.shape
users.head()

,ExternalID,BirthDate,Gender,Condition
0,4703,04/07/1965,F,Spina Bifida
1,4704,21/06/1940,F,COPD
2,4706,25/02/1931,F,Dementia
3,4707,18/09/1982,F,Brain Injury
4,4708,20/12/1931,F,NaN


In [15]:
#drop duplicates
users = users.drop_duplicates(subset = ['ExternalID'], keep='last')
users.shape

(680, 4)

In [16]:
#change BirthDate to date time
#users['BirthDate'] = pd.to_datetime(users['BirthDate'], format = "%m/%d/%y", dayfirst=False, utc=True)

users['BirthDate'] = pd.to_datetime(users['BirthDate'], errors='coerce', dayfirst=False, yearfirst=False, 
                   format="%d/%m/%Y", utc=False, infer_datetime_format=False, origin='unix')
users = users.set_index(pd.DatetimeIndex(users['BirthDate']))

#Remove erroneous entries
threshold = pd.to_datetime("2010-01-01", format="%Y-%m-%d", utc=False, origin='unix')
users = users[users['BirthDate'] < threshold]

users.head()

,ExternalID,BirthDate,Gender,Condition
BirthDate,,,,
1965-07-04,4703,1965-07-04,F,Spina Bifida
1940-06-21,4704,1940-06-21,F,COPD
1931-02-25,4706,1931-02-25,F,Dementia
1982-09-18,4707,1982-09-18,F,Brain Injury
1931-12-20,4708,1931-12-20,F,NaN


In [17]:
now = pd.to_datetime(date.today())

def getYears(start):
    diff = relativedelta(now, start)
    return diff.years

# Add a new column that contains the age of the user
users["Age"] = users['BirthDate'].map(getYears)

In [18]:
users.head()

,ExternalID,BirthDate,Gender,Condition,Age
BirthDate,,,,,
1965-07-04,4703,1965-07-04,F,Spina Bifida,53
1940-06-21,4704,1940-06-21,F,COPD,78
1931-02-25,4706,1931-02-25,F,Dementia,87
1982-09-18,4707,1982-09-18,F,Brain Injury,36
1931-12-20,4708,1931-12-20,F,NaN,86


In [19]:
users.shape

(672, 5)

In [20]:
#print all Cities, drop or group unneccessary
city = data.City.unique()
index = 1
for item in city:
    print(index, ': ', item)
    index += 1

1 :  Stirling
2 :  Ayr
3 :  Edinburgh
4 :  Dundee
5 :  Aberdeen
6 :  Dundee (Tayside)
7 :  test user
8 :  Glasgow
9 :  Arbroath (Tayside)


In [21]:
data = data.replace('Dundee (Tayside)', 'Dundee')

In [22]:
#count null values for City
sum(pd.isnull(data['City']))

0

In [23]:
location = data.Building.unique()
index = 1
for item in location:
    print(index, ': ', item)
    index += 1

1 :  Broom Ground
2 :  STH Ayshire St Leonards
3 :  Edinburgh West C@H
4 :  Broom Middle
5 :  Broom Supported Flats
6 :  Charleston
7 :  Aberdeen Raeden Court CH
8 :  LFC Main Building
9 :  Edinburgh Responder Service
10 :  Edinburgh
11 :  Complex Home 1
12 :  Edinburgh East C@H
13 :  Belses Gardens - Care Home
14 :  Glasgow C@H
15 :  Broom Houses
16 :  Aberdeen C@H
17 :  LFC Care at Home
18 :  Dundee
19 :  Arbroath


In [24]:
#see if they have locations so I can assign the city

pairLocation = {}
for index, row in (data.dropna()).iterrows():
    if row.Building in pairLocation:
        continue
    else:
        pairLocation[row.Building] = row.City

In [25]:
for index, row in data.iterrows():
    if row.City is None:
        if row.Building in pairLocation:
            # replace row with this row
            row.City = pairLocaton[row.Building]
        else: 
            continue      
    else: 
        continue

In [26]:
data

,Time,ExternalID,Role,Building,City,LinkTitle,LinkType
0,2018-10-29 10:55:00,5325,User,Broom Ground,Stirling,Sam Smith,Internet
1,2018-10-29 10:50:00,5325,User,Broom Ground,Stirling,Elton John,Internet
2,2018-10-29 10:49:00,5325,User,Broom Ground,Stirling,My Music,Category
3,2018-10-29 10:49:00,5325,User,Broom Ground,Stirling,Entertainment,Category
5,2018-10-29 10:41:00,5541,User,STH Ayshire St Leonards,Ayr,YouTube,Internet
6,2018-10-29 10:39:00,5541,User,STH Ayshire St Leonards,Ayr,YouTube,Internet
7,2018-10-29 10:38:00,5541,User,STH Ayshire St Leonards,Ayr,YouTube,Internet
15,2018-10-29 10:33:00,5541,User,STH Ayshire St Leonards,Ayr,YouTube,Internet
21,2018-10-29 10:12:00,5207,User,Edinburgh West C@H,Edinburgh,Candy Crush Soda,Internet
22,2018-10-29 10:12:00,5207,User,Edinburgh West C@H,Edinburgh,Single Player Games,Category
